# Initialize paths

In [ ]:
%cd ..
from popemp.tools import Nbd
Nbd('popemp')
%cd -

# Git config

In [ ]:
%cd ..
!nbdime config-git --enable
!git config filter.jupyternotebook.clean "jupyter nbconvert --stdin --stdout --to=notebook --ClearOutputPreprocessor.enabled=True --ClearMetadataPreprocessor.enabled=True --log-level=ERROR"
!git config filter.jupyternotebook.smudge cat
!git config filter.jupyternotebook.required true
!git config diff.jupyternotebook.command "git-nbdiffdriver diff --ignore-outputs --ignore-metadata --ignore-details"
%cd -

# Build all modules

In [ ]:
nbd.nb2mod('tools.ipynb')
nbd.nb2mod('data.ipynb')
nbd.nb2mod('analysis.ipynb')

# Build documentation site

In [ ]:
from popemp.tools import Nbd
nbd = Nbd('popemp')
nbd.build_docs()